In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

import matplotlib.pyplot as plt

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [7]:
lr = 1e-3
epochs = 15
batch_size = 100

In [8]:
# MNIST DATASET

mnist_train = dsets.MNIST(root="./MNIST_data/",
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)
mnist_test = dsets.MNIST(root="./MNIST_data/",
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

100.0%


Extracting ./MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST_data/MNIST\raw



102.8%


Extracting ./MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST_data/MNIST\raw



100.0%


Extracting ./MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST_data/MNIST\raw



112.7%

Extracting ./MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST_data/MNIST\raw




C:\Users\hwang\anaconda3\envs\Torch\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [10]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1,32,kernel_size=3, stride=1,padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
                        nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
                        nn.ReLU(),
                        nn.MaxPool2d(2)
        )
        self.fc = nn.Linear(7*7*64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1) # flatten
        out = self.fc(out)
        return out

In [16]:
model = CNN().to(device)

In [19]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [21]:
total_batch = len(data_loader)

for epoch in range(epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost/total_batch
    
    print('[Epoch: {}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

[Epoch: 1] cost = 0.2217486947774887
[Epoch: 2] cost = 0.05940479785203934
[Epoch: 3] cost = 0.04320048168301582
[Epoch: 4] cost = 0.034953709691762924
[Epoch: 5] cost = 0.029148761183023453
[Epoch: 6] cost = 0.025519737973809242
[Epoch: 7] cost = 0.02037414163351059
[Epoch: 8] cost = 0.017398305237293243
[Epoch: 9] cost = 0.014573552645742893
[Epoch: 10] cost = 0.011907888576388359
[Epoch: 11] cost = 0.010579823516309261
[Epoch: 12] cost = 0.008526796475052834
[Epoch: 13] cost = 0.008323385380208492
[Epoch: 14] cost = 0.00645016273483634
[Epoch: 15] cost = 0.006090404000133276
Learning Finished!


In [23]:
# Test session
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    pred = model(X_test)
    correct = torch.argmax(pred, 1) == Y_test
    acc = correct.float().mean()
    print('Accuray: {}'.format(acc.item()))

Accuray: 0.9874999523162842


In [25]:
pred[0]

tensor([ -7315.0962,  -3884.1919,  -2024.2725,   -450.2086,  -4173.7266,
         -2661.0933, -12943.0566,   5610.9775,  -3757.1892,   -203.3443],
       device='cuda:0')